In [11]:
#importamos librerías
import pandas as pd
from pandas import Series
import numpy as np
import re
import os
import funciones_analisis_join
#import src.funciones_analisis_join as fa 

In [17]:

#cargamos dataset con todos los vinos 
os.chdir("../Data")
df_vinos = pd.read_csv("df_vinos.csv")
df_vinos.name.head(15)
#vemos que tiene 4775 vinos , no hay duplicados y todos son valores unicos 

0                                 Gramona Ovum 2014
1           Ximénez-Spínola Pedro Ximénez Muy Viejo
2                             Chobeo de Peciña 2009
3                           Campo Viejo Tempranillo
4                                       Pingus 2016
5                                     Contador 2016
6                      Altos de Torona Godello 2016
7                              Manzanilla La Gitana
8                                       Fino Quinta
9                                 Manzanilla Solear
10                              Fino Maestro Sierra
11                                Octogenarius 2014
12    Monte Real Gran Reserva Edición Limitada 1998
13                         Tros de Clos Magnum 2014
14                  Tras da Canda Caiño Blanco 2016
Name: name, dtype: object

In [18]:
#ahora bamos a abrir el dataset de que comidas coinciden con que vino 
pairing_food_wines = pd.read_csv("pairing.csv")


In [19]:
#Observemos los valores de la tabla Wines, cuyo string tendra que coincidir con los valores de la columna name 
#de la tabla anterior ej .  Merlot Paris premium  y el string Merlot 
pairing_food_wines.Wines=pairing_food_wines.Wines.str.rstrip().str.lstrip()
pairing_food_wines.Wines

0            Cabernet Sauvignon
1                        Merlot
2                      Bordeaux
3                 red Zinfandel
4                         Rioja
                ...            
78                    ruby Port
79    Fino or Manzanilla Sherry
80              Sauvignon Blanc
81                     Sancerre
82    Vouvray or white Bordeaux
Name: Wines, Length: 83, dtype: object

In [20]:
#vemos que en algunas filas hay dos tipos de vino , por ej Vouvray or white Bordeaux
#vamos a dividirlo con un split y explotarlo 

data_f_w=pd.concat(   [Series(row['Food'], row['Wines'].split('or '))  for _, row in pairing_food_wines.iterrows()]).reset_index()



In [21]:
#unimos dataframe con categorias de vinos y comidas que encajan con el dataframe del catalogo con nombres de vinos
q = pd.concat([df_vinos.name,data_f_w], axis=1)
q

,name,index,0
0,Gramona Ovum 2014,Cabernet Sauvignon,Beef
1,Ximénez-Spínola Pedro Ximénez Muy Viejo,Merlot,Beef
2,Chobeo de Peciña 2009,Bordeaux,Beef
3,Campo Viejo Tempranillo,red Zinfandel,Beef
4,Pingus 2016,Rioja,Beef
...,...,...,...
4771,Alvear PX Solera 1927,NaN,NaN
4772,Alabaster 2016,NaN,NaN
4773,Cantos del Diablo 2016,NaN,NaN
4774,La Mujer Cañón 2016,NaN,NaN


In [22]:
#quitamos indices
q.reset_index

<bound method DataFrame.reset_index of                                          name               index     0
0                           Gramona Ovum 2014  Cabernet Sauvignon  Beef
1     Ximénez-Spínola Pedro Ximénez Muy Viejo              Merlot  Beef
2                       Chobeo de Peciña 2009            Bordeaux  Beef
3                     Campo Viejo Tempranillo       red Zinfandel  Beef
4                                 Pingus 2016               Rioja  Beef
...                                       ...                 ...   ...
4771                    Alvear PX Solera 1927                 NaN   NaN
4772                           Alabaster 2016                 NaN   NaN
4773                   Cantos del Diablo 2016                 NaN   NaN
4774                      La Mujer Cañón 2016                 NaN   NaN
4775                           Moncerbal 2016                 NaN   NaN

[4776 rows x 3 columns]>

In [23]:
#renombramos columnas
q.rename(columns={"name": "wine_name", "index": "category_wine",0: "category_food"}, inplace=True)

In [24]:
q

,wine_name,category_wine,category_food
0,Gramona Ovum 2014,Cabernet Sauvignon,Beef
1,Ximénez-Spínola Pedro Ximénez Muy Viejo,Merlot,Beef
2,Chobeo de Peciña 2009,Bordeaux,Beef
3,Campo Viejo Tempranillo,red Zinfandel,Beef
4,Pingus 2016,Rioja,Beef
...,...,...,...
4771,Alvear PX Solera 1927,NaN,NaN
4772,Alabaster 2016,NaN,NaN
4773,Cantos del Diablo 2016,NaN,NaN
4774,La Mujer Cañón 2016,NaN,NaN


In [25]:
#para evtar problemas con nulos los rellenamos con 'unknown'
q = q.fillna('unknown')
q

,wine_name,category_wine,category_food
0,Gramona Ovum 2014,Cabernet Sauvignon,Beef
1,Ximénez-Spínola Pedro Ximénez Muy Viejo,Merlot,Beef
2,Chobeo de Peciña 2009,Bordeaux,Beef
3,Campo Viejo Tempranillo,red Zinfandel,Beef
4,Pingus 2016,Rioja,Beef
...,...,...,...
4771,Alvear PX Solera 1927,unknown,unknown
4772,Alabaster 2016,unknown,unknown
4773,Cantos del Diablo 2016,unknown,unknown
4774,La Mujer Cañón 2016,unknown,unknown


In [26]:
#hacemos lista las categorías de vinos. DEspues intentaremos ver que categorías de vinos matchean con los nombres de vinos 
#ej Categoría de vino: Merlot , nombre vino: Merlot parisien 2017  . Como el nombre contiene Merlot matchean
#como la columna de categorías esta asociada a una columna de comidas que pegan con esas categorías, entonces si categoría y vino 
#matchean que se ponga una columna con TRue y False.
lista=list(q['category_wine'])
lista

['Cabernet Sauvignon',
 'Merlot',
 'Bordeaux',
 'red Zinfandel',
 'Rioja',
 'Ribera del Duero',
 'Châteauneuf-du-Pâpe',
 'Hermitage',
 'Côte-Rôtie',
 'Australian Shiraz',
 'Super Tuscan',
 'Barbaresco ',
 'Barolo',
 'Bordeaux (especially Médoc)',
 'Rioja',
 'Ribera del Duero',
 'Châteauneuf-du-Pâpe',
 'Barbaresco ',
 'Barolo PorkRiesling',
 'Cru Beaujolais',
 'Rioja',
 'Côtes du Rhône ',
 'New World ChardonnayRiesling',
 'Cru Beaujolais',
 'Rioja',
 'Côtes du Rhône ',
 'New World ChardonnayBBQ',
 'Côtes du Rhône',
 'Châteauneuf-du-Pâpe',
 'Shiraz',
 'Petite Syrah ',
 'red Zinfandel',
 'California ',
 'Australian Chardonnay',
 'Riesling',
 'dry Vouvray',
 'white Burgundy',
 'red Burgundy',
 'Gigondas',
 'Côtes du Rhône',
 'Cabernet Sauvignon',
 'Cabernet Franc',
 'Merlot',
 'red Zinfandel ',
 'Valpolicella',
 'Red Burgundy',
 'Pinot Noir ',
 'Rioja',
 'Pinot Grigio',
 'Vernaccia',
 'Barbera',
 'Dolcetto',
 'Chianti',
 'Pinot Noir ',
 'Cabernet Franc',
 'Chianti',
 'Morellino di Scansano

In [27]:
def find_value_column(row):
    for keyword in row['category_wine'].lower():
        return keyword in row.wine_name.lower()
    return False


q.new=q.apply(find_value_column, axis=1)

/home/usuario/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  import sys


In [30]:
q.new

0       False
1        True
2        True
3        True
4       False
        ...  
4771    False
4772    False
4773    False
4774     True
4775    False
Length: 4776, dtype: bool

In [29]:
q.new.value_counts()

False    2605
True     2171
dtype: int64

In [31]:
#concatenamos la columna true false con el dataframe que contiene los vinos , las categorias y las comidas
q = pd.concat([q,q.new], axis=1)
q

,wine_name,category_wine,category_food,0
0,Gramona Ovum 2014,Cabernet Sauvignon,Beef,False
1,Ximénez-Spínola Pedro Ximénez Muy Viejo,Merlot,Beef,True
2,Chobeo de Peciña 2009,Bordeaux,Beef,True
3,Campo Viejo Tempranillo,red Zinfandel,Beef,True
4,Pingus 2016,Rioja,Beef,False
...,...,...,...,...
4771,Alvear PX Solera 1927,unknown,unknown,False
4772,Alabaster 2016,unknown,unknown,False
4773,Cantos del Diablo 2016,unknown,unknown,False
4774,La Mujer Cañón 2016,unknown,unknown,True


In [33]:
#renombramos
q.rename(columns={0: "check"}, inplace=True)


In [35]:
#aplicamos las funciones transformadoras. Si category_food es == a True que deje el nombre de la comida , sino 
#unknown
food_choose=q[['category_food','check']].apply(transf2,axis=1)


#aplicamos las funciones transformadoras. Si wine_name es == a True que deje el nombre del vino , sino 
#unmatch

wine_choose=q[['wine_name','check']].apply(transf3,axis=1)

In [36]:
#concatenamos resultado y renombramos
q = pd.concat([food_choose,wine_choose], axis=1)
q.rename(columns={1: "Wines",0:'Food_pair'}, inplace=True)
q

,Food_pair,Wines
0,unmatch,unmatch
1,Beef,Ximénez-Spínola Pedro Ximénez Muy Viejo
2,Beef,Chobeo de Peciña 2009
3,Beef,Campo Viejo Tempranillo
4,unmatch,unmatch
...,...,...
4771,unmatch,unmatch
4772,unmatch,unmatch
4773,unmatch,unmatch
4774,unknown,La Mujer Cañón 2016


In [37]:
#filtramos para quitarnos los unmatch y los unknown

q=q[(q['Wines']!='unmatch') & (q['Food_pair']!='unknown')]

#ya tenemos nuestra lista de nombres de vinos del catalogo asociados a comidas. Se nos ha reducido bastante
#por lo que lo suyo es aumentar la lista de vinos y de categorías d evinos con comida a futuros.
q.head(20)

,Food_pair,Wines
1,Beef,Ximénez-Spínola Pedro Ximénez Muy Viejo
2,Beef,Chobeo de Peciña 2009
3,Beef,Campo Viejo Tempranillo
5,Beef,Contador 2016
9,Beef,Manzanilla Solear
10,Beef,Fino Maestro Sierra
14,Lamb,Tras da Canda Caiño Blanco 2016
15,Lamb,Fernando de Castilla Classic Manzanilla
16,Lamb,Fernando de Castilla Classic Oloroso
21,Lamb,El Sacramento 2014


In [38]:
#guardamos
q.to_csv('dataset_final.csv')